## Data Preparation
To begin this exploratory analysis, first use `matplotlib` to import libraries and define functions for plotting the data.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import os
import shutil
# from keras.applications.inception_v3 import InceptionV3
# from keras.applications.inception_v3 import preprocess_input, decode_predictions
# from keras.applications.resnet import ResNet152
# from keras.applications.resnet import preprocess_input, decode_predictions
from keras.applications.nasnet import NASNetLarge
from keras.applications.nasnet import preprocess_input, decode_predictions
# from keras.applications.inception_resnet_v2 import InceptionResNetV2
# from keras.applications.inception_resnet_v2 import preprocess_input, decode_predictions
# from keras.applications.xception import Xception
# from keras.applications.xception import preprocess_input, decode_predictions
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from collections import Counter
from keras.layers import Conv2D, GlobalAveragePooling2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential
from keras import optimizers
from sklearn.preprocessing import LabelEncoder 
from keras.utils import to_categorical
import math

In [ ]:
x_val = [0, .1, .2]
xception_article = [66.38, 75.56, 80.96]
InceptionV3_article = [61.81, 71.85, 78.15]
Resnet50_article = [67.84, 76.73, 82.21]
Resnet50v2_article = [66.78, 76.17, 81.97]
Resnet152_article = [ 68.66, 77.44, 82.99]
InceptionResnet_article = [ 62.78, 71.93, 78.27]
NasnetLarge_article = [ 68.48, 77.40, 82.71]


xception_gender = [ 56.40, 68.27, 75.98]
InceptionV3_gender = [ 53.77, 66.62, 75.04]
Resnet50_gender = [63.67, 74.24, 81.02]
Resnet50v2_gender = [ 60.26, 71.74, 79.32]
Resnet152_gender = [ 63.94, 74.75, 81.39]
InceptionResnet_gender = [ 53.68, 65.49, 73.73]
NasnetLarge_gender = [ 54.14, 66.86, 75.81]


xception_gender_article = [ 42.41, 54.46, 63.16]
InceptionV3_gender_article = [ 38.41, 51.04, 60.23]
Resnet50_gender_article = [ 47.03, 58.92, 67.50 ]
Resnet50v2_gender_article = [ 44.00, 56.42, 65.50]
Resnet152_gender_article = [ 47.80, 59.98, 68.48]
InceptionResnet_gender_article = [ 38.61, 49.99, 59.01]
NasnetLarge_gender_article = [ 43.75, 56.12, 64.91]



plt.figure(figsize=(21, 24))
plt.subplot(3, 1, 1)
plt.plot(x_val, xception_article, label='Xception')
plt.plot(x_val,InceptionV3_article, label='InceptionV3')
plt.plot(x_val,Resnet50_article, label='Resnet50')
plt.plot(x_val,Resnet50v2_article, label='Resnet50v2')
plt.plot(x_val,Resnet152_article, label='Resnet152')
plt.plot(x_val,InceptionResnet_article, label='InceptionResnet')
plt.plot(x_val,NasnetLarge_article, label='NasnetLarge')

plt.legend(loc='lower right')
plt.ylabel('Precision')
plt.title('Precision for article type')

plt.subplot(3, 1, 2)
plt.plot(x_val, xception_gender, label='Xception')
plt.plot(x_val, InceptionV3_gender, label='InceptionV3')
plt.plot(x_val, Resnet50_gender, label='Resnet50')
plt.plot(x_val, Resnet50v2_gender, label='Resnet50v2')
plt.plot(x_val, Resnet152_gender, label='Resnet152')
plt.plot(x_val, InceptionResnet_gender, label='InceptionResnet')
plt.plot(x_val, NasnetLarge_gender, label='NasnetLarge')

plt.legend(loc='lower right')
plt.ylabel('Precision')
plt.title('Precision for gender')


plt.subplot(3, 1, 3)
plt.plot(x_val,xception_gender_article, label='Xception')
plt.plot(x_val,InceptionV3_gender_article, label='InceptionV3')
plt.plot(x_val,Resnet50_gender_article, label='Resnet50')
plt.plot(x_val,Resnet50v2_gender_article, label='Resnet50v2')
plt.plot(x_val,Resnet152_gender_article, label='Resnet152')
plt.plot(x_val,InceptionResnet_gender_article, label='InceptionResnet')
plt.plot(x_val,NasnetLarge_gender_article, label='NasnetLarge')

plt.legend(loc='lower right')
plt.ylabel('Precision')
plt.title('Precision for gender and article type')
plt.xlabel('Tolerance')
plt.show()

In [ ]:
DATASET_PATH = "/kaggle/input/fashion-product-images-dataset/fashion-dataset/fashion-dataset/"
print(os.listdir(DATASET_PATH))

In [ ]:
# df = pd.read_csv(DATASET_PATH + "styles.csv", nrows=1000, error_bad_lines=False)
df = pd.read_csv(DATASET_PATH + "styles.csv", error_bad_lines=False)
# df = df.groupby('articleType').filter(lambda x: len(x) >= 650);
# df = df.groupby('articleType').head(11)
# delete_row = df[df["id"]==39403].index
# df = df.drop(delete_row)
# delete_row = df[df["id"]==12347].index
# df = df.drop(delete_row)
# delete_row = df[df["id"]==39410].index
# df = df.drop(delete_row)
# delete_row = df[df["id"]==39425].index
# df = df.drop(delete_row)
# delete_row = df[df["id"]==39401].index
# df = df.drop(delete_row)
df['image'] = df.apply(lambda row: str(row['id']) + ".jpg", axis=1)
df = df.reset_index(drop=True)
df.head(10)
df.articleType.value_counts()

In [ ]:
df[df["id"]==12347]

In [ ]:
import cv2
def plot_figures(figures, nrows = 1, ncols=1,figsize=(5, 5)):
    """Plot a dictionary of figures.

    Parameters
    ----------
    figures : <title, figure> dictionary
    ncols : number of columns of subplots wanted in the display
    nrows : number of rows of subplots wanted in the figure
    """

    fig, axeslist = plt.subplots(ncols=ncols, nrows=nrows,figsize=figsize)
    for ind,title in enumerate(figures):
        axeslist.ravel()[ind].imshow(cv2.cvtColor(figures[title], cv2.COLOR_BGR2RGB))
        axeslist.ravel()[ind].set_title(title)
        axeslist.ravel()[ind].set_axis_off()
    plt.tight_layout() # optional
    
def img_path(img):
    return DATASET_PATH+"images/"+img

def load_image(img, resized_fac = 1):
    img     = cv2.imread(img_path(img))
    w, h, _ = img.shape
    resized = cv2.resize(img, (int(h*resized_fac), int(w*resized_fac)), interpolation = cv2.INTER_AREA)
    return resized

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# generation of a dictionary of (title, images)
figures = {'im'+str(i): load_image(row.image) for i, row in df.sample(6).iterrows()}
# plot of the images in a figure, with 2 rows and 3 columns
plot_figures(figures, 2, 3)

The Dataset is made up of different items that can be found in a marketplace. The idea is to use embeddings to search for similarity and find similar items just using the image.

## Use Pre-Trained Model to Recommendation

In [ ]:
import tensorflow as tf
import keras
from keras import Model
from keras.preprocessing import image
from keras.layers import GlobalMaxPooling2D
tf.__version__

In [ ]:
# # detect and init the TPU
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)

# # instantiate a distribution strategy
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
# Input Shape
img_width, img_height, _ = 331, 331, 3 #load_image(df.iloc[0].image).shape

# Pre-Trained Model
base_model = NASNetLarge(weights='imagenet', 
                      include_top=False, 
                      input_shape = (img_width, img_height, 3))
base_model.trainable = False

# Add Layer Embedding
model = keras.Sequential([
    base_model,
    GlobalMaxPooling2D()
])

model.summary()

In [ ]:
i=0;
def get_embedding(model, img_name):
    # Reshape
#     if os.path.exists(img_path(img_name)):
        img = image.load_img(img_path(img_name), target_size=(img_width, img_height))
        # img to Array
        x   = image.img_to_array(img)
        # Expand Dim (1, w, h)
        x   = np.expand_dims(x, axis=0)
        # Pre process Input
        x   = preprocess_input(x)
        global i
        i=i+1
        print(i)
        return model.predict(x).reshape(-1)
#     else:
#         return []

Get item Embedding

In [ ]:
emb = get_embedding(model, df.iloc[0].image)
emb.shape

In [ ]:
img_array = load_image(df.iloc[1].image)
plt.imshow(cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB))
print(img_array.shape)
print(emb)

In [ ]:
df.shape

Get Embedding for all itens in dataset

In [ ]:
%%time
#import swifter

# Parallel apply
df_sample      = df#.sample(10)
map_embeddings = df_sample['image'].apply(lambda img: get_embedding(model, img))
df_embs        = map_embeddings.apply(pd.Series)

print(df_embs.shape)
df_embs.head()

## Compute Similarity Between Items

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise_distances.html
import time
from sklearn.metrics.pairwise import pairwise_distances
start_time = time.time();

# Calcule DIstance Matrix
# reduce_mem_usage(df_embs)
cosine_sim = 1-pairwise_distances(df_embs, metric='cosine')
cosine_sim[:4, :4]
end_time = time.time();
spent_time = end_time - start_time;
print(spent_time)
# reduce_mem_usage(cosine_sim )

#### Recommender Similar Items

In [ ]:
indices = pd.Series(range(len(df_embs)), index=df_embs.index)

In [ ]:
from scipy.spatial.distance import cosine
# Function that get movie recommendations based on the cosine similarity score of movie genres
def get_recommender(idx, df, options, top_n = 5):
    sim_idx    = indices[idx]
    
    filteredIndices = set();
    
    if len(options) > 0:
        it = 0
        for column in options:
            if it == 0:
                filteredIndices = set(df.index[df[column]== options[column]].tolist())
            else:
                filteredIndices = filteredIndices & set(df.index[df[column]== options[column]].tolist())
            it = it + 1
            
        filteredIndices = list(filteredIndices)
#         print(len(filteredIndices) )
    else:
        filteredIndices = list(indices.index)
#     sim_scores =[];
#     for i in range(len(df_embs)):
#         sim_scores[i] = 1-cdist(df_embs[df_embs.index == sim_idx],df_embs[i], metric='cosine')
    cosine_sim = [ 1 - cosine(df_embs.iloc[sim_idx], df_embs.iloc[i]) for i in range(len(df_embs))]
    sim_scores = list(enumerate(cosine_sim))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1: ]
    
    idx_rec = []
    idx_sim = []
    count_sim = 0
    
    for i in sim_scores:
        if i[0] in filteredIndices:
            idx_rec.append(i[0])
            idx_sim.append(i[1])
            count_sim = count_sim + 1
            
            if count_sim >= top_n:
                break
            
    
    idx_rec = indices.iloc[idx_rec].index
    plt.imshow(cv2.cvtColor(load_image(df.iloc[idx].image), cv2.COLOR_BGR2RGB))
    
    
    if(len(idx_rec) <= 0):
        print('No matching item found for', options)
    
    plot_row = math.ceil(len(idx_sim)/3)
    plot_col = min(3, len(idx_sim))
        # Plot
        #===================
        # generation of a dictionary of (title, images)
    figures = {'im'+str(i): load_image(row.image) for i, row in df.loc[idx_rec].iterrows()}
        # plot of the images in a figure, with 2 rows and 3 columns
    if len(figures):
        plot_figures(figures, plot_row, plot_col)
#     return idx_rec

In [ ]:
import time
def get_recommender_for_precision(idx, df, options, top_n = 5):
    start_time = time.time();
    sim_idx    = indices[idx]
    
    filteredIndices = set();
    
    if len(options) > 0:
        it = 0
        for column in options:
            if it == 0:
                filteredIndices = set(df.index[df[column]== options[column]].tolist())
            else:
                filteredIndices = filteredIndices & set(df.index[df[column]== options[column]].tolist())
            it = it + 1
            
        filteredIndices = list(filteredIndices)
#         print(len(filteredIndices) )
    else:
        filteredIndices = list(indices.index)
#     sim_scores =[];
#     for i in range(len(df_embs)):
#         sim_scores[i] = 1-cdist(df_embs[df_embs.index == sim_idx],df_embs[i], metric='cosine')
#     cosine_sim = [ 1 - cosine(df_embs.iloc[sim_idx], df_embs.iloc[i]) for i in range(len(df_embs))]
#     sim_scores = list(enumerate(cosine_sim))
    sim_scores = list(enumerate(cosine_sim[sim_idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1: ]
    
    idx_rec = []
    idx_sim = []
    count_sim = 0
    
    for i in sim_scores:
        if i[0] in filteredIndices:
            idx_rec.append(i[0])
            idx_sim.append(i[1])
            count_sim = count_sim + 1
            
            if count_sim >= top_n:
                break
            
    
    idx_rec = indices.iloc[idx_rec].index
#     plt.imshow(cv2.cvtColor(load_image(df.iloc[idx].image), cv2.COLOR_BGR2RGB))
    
    
    if(len(idx_rec) <= 0):
        print('No matching item found for', options)
    
#     plot_row = math.ceil(len(idx_sim)/3)
#     plot_col = min(3, len(idx_sim))
#         # Plot
#         #===================
#         # generation of a dictionary of (title, images)
#     figures = {'im'+str(i): load_image(row.image) for i, row in df.loc[idx_rec].iterrows()}
#         # plot of the images in a figure, with 2 rows and 3 columns
#     if len(figures):
#         plot_figures(figures, plot_row, plot_col)
    end_time = time.time();
    return idx_rec

In [ ]:
# time_needed,id_rec = get_recommender_for_precision(7,df,{},10)
# print(time_needed)

In [ ]:
# precision = tp/tp+fp
def get_precision(df,top_n =10,tolarence=.8):
    tp = len(df);
    for i in range(len(df)):
#         if i == 12347 or i == 39401 or i == 39403 or i == 39410 or i == 39425:
#             continue;
        id_rec= get_recommender_for_precision(i, df, {}, top_n);
        c=0;
        for j in range(len(id_rec)): 
            if df.iloc[i].gender == df.iloc[id_rec[j]].gender: # and df.iloc[i].articleType == df.iloc[id_rec[j]].articleType
                c=c+1;
        success_rate = c/top_n;
        if success_rate < tolarence:
            tp = tp -1;
    return tp/len(df);

In [ ]:
df.iloc[1].gender

# Precision Calculation 

In [ ]:
prec = get_precision(df,10,1)
print(prec)
df.iloc[1].articleType

In [ ]:
base_color_filter = df['baseColour']=='Green'

In [ ]:
len(df_embs)

In [ ]:
df.iloc[19572].gender 

In [ ]:
# Idx Item to Recommender
idx_ref = 7831

# Recommendations
get_recommender(idx_ref, df, { }, top_n = 6)
# get_recommender(idx_ref, df, { "gender":df.iloc[idx_ref].gender }, top_n = 6)
# get_recommender(idx_ref, df, { "articleType":df.iloc[idx_ref].articleType }, top_n = 6)

In [ ]:
df[df.index == 22]

In [ ]:
df[df.subCategory == 'Shoe'][df.baseColour == 'Black']

**Search for items similar to the reference to recommend. Apparently it's working!**

In [ ]:
idx_ref = 8921

# Recommendations
idx_rec, idx_sim = get_recommender(idx_ref, df, {}, top_n = 6)

# Plot
#===================
plt.imshow(cv2.cvtColor(load_image(df.iloc[idx_ref].image), cv2.COLOR_BGR2RGB))

# generation of a dictionary of (title, images)
figures = {'im'+str(i): load_image(row.image) for i, row in df.loc[idx_rec].iterrows()}
# plot of the images in a figure, with 2 rows and 3 columns
plot_figures(figures, 2, 3)

In [ ]:
idx_ref = 987

# Recommendations
idx_rec, idx_sim = get_recommender(idx_ref, df, {}, top_n = 6)

# Plot
#===================
plt.imshow(cv2.cvtColor(load_image(df.iloc[idx_ref].image), cv2.COLOR_BGR2RGB))

# generation of a dictionary of (title, images)
figures = {'im'+str(i): load_image(row.image) for i, row in df.loc[idx_rec].iterrows()}
# plot of the images in a figure, with 2 rows and 3 columns
plot_figures(figures, 2, 3)

In [ ]:
idx_ref = 3524

# Recommendations
idx_rec, idx_sim = get_recommender(idx_ref, df, {}, top_n = 6)

# Plot
#===================
plt.imshow(cv2.cvtColor(load_image(df.iloc[idx_ref].image), cv2.COLOR_BGR2RGB))

# generation of a dictionary of (title, images)
figures = {'im'+str(i): load_image(row.image) for i, row in df.loc[idx_rec].iterrows()}
# plot of the images in a figure, with 2 rows and 3 columns
plot_figures(figures, 2, 3)

In [ ]:
# precision = tp/tp+fp
def get_precision(df,top_n =10):
    tp = len(df);
    for i in range(len(df)):
        id_rec= get_recommender(i, df, {}, top_n);
        for j in range(len(id_rec)):
            if df.iloc[i].articleType != df.iloc[id_rec[j]].articleType :
                tp = tp -1;
                break;
    return tp/len(df);